### defining the transportation unit class
<li>a transportation unit has a location (latitude and longitude)<li>person capacity: maximum of number of person that can be moved<li>battery_state: what percentage of battery is still full<li>battery_capacity: number of kwh when fully charged<li>avg_km_per_kwh : number of km driver per kilometer<li>state: possible values 1 = 'charging', 2 = 'discharging', 3 = 'stationary' (doing nothing), 4 = 'person transport', 5 = 'package transport

a tranportation unit will constantly make a request/decision about what to do next:
charge, discharge, transport person, transport package, do nothing (or maybe move empty to area with highest chance of picking up person/package)




In [1]:
import ipynb.fs.defs.helper_functions as helper_functions

In [2]:
class Tu():
    _registry = []
    
    def __init__(self, name, lat, lon, person_capacity = 4, battery_state=0.2, battery_capacity=300, 
                 avg_km_per_kwh=7, state=3, km_hour = 150
                ):
        "assumes name is string"
        self._registry.append(self)
        self.name = name
        self.lat = lat
        self.lon = lon
        self.person_capacity = person_capacity
        self.battery_state = battery_state
        self.battery_capacity = battery_capacity
        self.avg_km_per_kwh = avg_km_per_kwh
        self.state = state
        self.km_hour = km_hour
    
    def get_name(self):
        return self.name
    
    def get_location(self):
        return(self.lat, self.lon)

    def get_battery_state(self):
        return self.battery_state
    
    def get_state(self):
        return self.state
    
    def get_radius(self):
        return self.battery_capacity * self.battery_state * self.avg_km_per_kwh
    
    def get_km_hour(self):
        return self.km_hour
    
    def __str__(self):
        return self.get_name() \
        + ' at lat:' + str(tu1.get_location()[0]) + ' and lon:' + str(self.get_location()[1])\
        +' radius left:'  + str(self.get_radius())
        
    def update_location(self, lat, lon):
        self.lon = lon
        self.lat = lat
        
    def update_state(self, state):
        self.state = state
        
#  where should we calculate the decrease in capacity after a trip has been completed
#  is this a method based upon the new location? 
#  where do we calculate the charging of the battery

    def update_capacity(self, capacity):
        self.capacity = capacity

# next action defines what the car will do next      
# maybe we can also use reinforcement learning with multi arm bandit logic where the arms are the possible actions
#     def next_action(self):
#         update state
#         update time
#         update capacity

# I guess a TU will exist in a certain time, how do we 'simulate'  this
#     def update_time(self, time):
#         self.time = time

    

### transportation_request class

In [3]:
import datetime

class Tr():
    
    def __init__(self, start_lat, start_lon, end_lat, end_lon,  request_time, transport_type, request_id, factor_ride=1.5):
        self.start_lat = float(start_lat)
        self.start_lon = float(start_lon)
        self.end_lat = float(end_lat)
        self.end_lon = float(end_lon)
        self.request_id = request_id
        self.transport_type = transport_type
        self.request_time = request_time
        self.factor_ride = factor_ride
    
    def get_distance_ride(self):
        return helper_functions.distance((self.start_lat, self.start_lon),(self.end_lat,self.end_lon)) * self.factor_ride
    
    def get_request_id(self):
        return self.request_id
    
    def get_request_time(self):
        return self.request_time
    
    def get_start_loc(self):
        return(self.start_lat, self.start_lon)
    
    def get_distance_to_tu(self, tu_location):
        #location of transportation unit
        return helper_functions.distance((tu_location[0], tu_location[1]), (self.start_lat, self.start_lon)) * self.factor_ride
    
    def __str__(self):
             return 'id:' + str(self.get_request_id()) + ' requests a transport at ' \
        + str(self.request_time) +' for ' \
        + str(round(self.get_distance_ride(),3)) + " KM" \
        + ' at '+ str(self.get_start_loc())
        #+ datetime.time.strftime("%Y-%m-%d %H:%M:%S", self.request_time)
        

### iterating over all classses?
https://stackoverflow.com/questions/739882/iterating-over-object-instances-of-a-given-class-in-python
is this a good way to find open requests, all open request could iterate over the available cars and once the request is succesfully fullfilled (in tangle) written to the database

wat moet er centraal staan in aanvraag.
wanneer er b.v. een auto beschikbaar is die nog 10km van aanvrager weg is en er op 500 meter een auto beschikbaar komt over 30 seconde omdat deze dan klaar is met het vervoeren van zijn huidige passagier. wat is dan het beste.

wanneer we de auto centraal zetten gaat deze zoeken naar een passagier, de auto vindt de dichtstbijzijnde passagier die staat te wachten op 10KM, deze twee gaan dan een transactie aan en de auto krijgt de state:'onderweg' en de passagier krijgt de state: 'wordt opgepikt' 

wanneer de passagier centraal staat kijkt deze naar alle auto's,  

auto's zonder passagier: rechtstreekse route

auto's met passagier: indirecte route (huidige locatie van auto naar eindbestemming en van eindbestemming naar passagier)

qua optimalisatie lijkt optie met passagier centraal het handigst

In [4]:
import json
import datetime

with open('../ridelisttest.json') as json_data:
    ride_list = json.load(json_data)
    ride_list.sort()

In [5]:
def generate_ride(tu, tr, ride_dict, distance_tu_tr, current_ts):
    tu.update_state(4)
    waiting_time = (current_ts-tr.get_request_time())  + (distance_tu_tr/tu.get_km_hour()*3600)
    ride_dict[tr.get_request_id()] = [tu.get_name(), distance_tu_tr, tr.get_distance_ride(), waiting_time]
    available_ts = int(current_ts + (distance_tu_tr/tu.get_km_hour()*3600) + (tr.get_distance_ride()/tu.get_km_hour()*3600))
    available_tu_ts[tu.get_name()] = [available_ts, tr.end_lat, tr.end_lon]
    del tr
    return ride_dict, available_tu_ts

    

In [6]:
def finish_ride(tu, end_lat, end_lon):
    tu.update_state(3)
    tu.update_location(end_lat, end_lon)

In [7]:
#input is 1 tr
def find_tu(tr):
    distance_dict = {}
    for tu in Tu._registry:
        if tu.get_state() == 3:
            distance_dict[tu.get_name()] = round(tr.get_distance_to_tu(tu.get_location()),2)
    if len(distance_dict.keys()) > 0 :
        match_tu_str = min(distance_dict, key=distance_dict.get)
        return match_tu_str, distance_dict[match_tu_str]
    

In [8]:
#initiate 2 tu's op t = 0 in groningen
tu1 = Tu(name='tu1', lat=53.2194, lon=6.5665, person_capacity = 4, battery_state=0.2, battery_capacity=85, 
                 avg_km_per_kwh=7, state=3)

tu2 = Tu(name='tu2', lat=53.4194, lon=6.6665, person_capacity = 4, battery_state=0.8, battery_capacity=70, 
                 avg_km_per_kwh=5.92, state=3)

tu3 = Tu(name='tu3', lat=53.4184, lon=6.6665, person_capacity = 4, battery_state=0.8, battery_capacity=70, 
                 avg_km_per_kwh=5.92, state=3)

tu4 = Tu(name='tu4', lat=53.4174, lon=6.6665, person_capacity = 4, battery_state=0.8, battery_capacity=70, 
                 avg_km_per_kwh=5.92, state=3)

tu5 = Tu(name='tu5', lat=53.4164, lon=6.6665, person_capacity = 4, battery_state=0.8, battery_capacity=70, 
                 avg_km_per_kwh=5.92, state=3)
print(tu1)
print(tu2)
    
ride_list_test = ride_list[0:100]
ride_dict = {}
available_tu_ts = {}

tu1 at lat:53.2194 and lon:6.5665 radius left:119.0
tu2 at lat:53.2194 and lon:6.6665 radius left:331.52


In [9]:
datum = datetime.datetime(2018, 4, 9, 0, 0, 0)
datum = int(datum.timestamp())

waiting_list = []
for i in range(24*60*60):
    current_ts = datum + i
    if len(ride_list_test) > 0 and current_ts >= ride_list_test[0][0]:
        # now the logic for the ride starts for selecting a transportation unit
        tr = Tr(ride_list_test[0][3], ride_list_test[0][4], ride_list_test[0][5], ride_list_test[0][6], ride_list_test[0][0], 
               1, ride_list_test[0][2])
        waiting_list.append(tr)
        for tr in waiting_list:
            try:
                #print(current_ts, 'try1')
                match_tu, distance_tu_tr = find_tu(tr)
                print(distance_tu_tr)
                print(current_ts, 'try2')
                ride_dict, available_tu_ts = generate_ride(locals().get(match_tu), tr, ride_dict, distance_tu_tr, current_ts)
                print(available_tu_ts)
                print(current_ts, 'try3')
                waiting_list.remove(tr)
                del match_tu, distance_tu_tr
            except:
                print('JOOST')
                pass
        del ride_list_test[0]
        #kijk of timestamp overeenkomt met einde rit tijd die terugkomt uit generate_ride
    available_tu_ts2 = available_tu_ts.copy()
    for tu, ts in available_tu_ts2.items():
        if int(current_ts) == int(ts[0]):
            print(tu,ts, "MAKE RIDE AVAILABLE!!!!!")
            finish_ride(locals().get(tu), ts[1], ts[2])
            available_tu_ts.pop(tu, None)

2.22
1523233350 try2
{'tu1': [1523233564, 53.23188273, 6.522460818]}
1523233350 try3
tu1 [1523233564, 53.23188273, 6.522460818] MAKE RIDE AVAILABLE!!!!!
5.84
1523236544 try2
{'tu1': [1523236853, 53.23081805, 6.623039211]}
1523236544 try3
tu1 [1523236853, 53.23081805, 6.623039211] MAKE RIDE AVAILABLE!!!!!
8.61
1523237350 try2
{'tu1': [1523237703, 53.20951859, 6.580977812]}
1523237350 try3
tu1 [1523237703, 53.20951859, 6.580977812] MAKE RIDE AVAILABLE!!!!!
6.05
1523238433 try2
{'tu1': [1523238737, 53.21765402, 6.575561271]}
1523238433 try3
34.61
1523238660 try2
{'tu1': [1523238737, 53.21765402, 6.575561271], 'tu5': [1523239612, 53.18884102, 6.552144116]}
1523238660 try3
tu1 [1523238737, 53.21765402, 6.575561271] MAKE RIDE AVAILABLE!!!!!
1.56
1523239412 try2
{'tu5': [1523239612, 53.18884102, 6.552144116], 'tu1': [1523239492, 53.2028864, 6.559533838]}
1523239412 try3
32.96
1523239456 try2
{'tu5': [1523239612, 53.18884102, 6.552144116], 'tu1': [1523239492, 53.2028864, 6.559533838], 'tu4': [

In [10]:
len(ride_dict)

95

In [ ]:
ride_list_test[4]

In [ ]:
available_tu_ts.items()

In [ ]:

print(available_tu_ts)
for tu, ts in available_tu_ts.items():
        if 1523238226 in ts:
            finish_ride(locals().get(tu), ts[1], ts[2])

In [ ]:
tr = Tr(ride_list_test[0][3], ride_list_test[0][4], ride_list_test[0][5], ride_list_test[0][6], ride_list_test[0][0], 
               1, ride_list_test[0][2])
waiting_list.append(tr)


In [ ]:
current_ts = 1523233350
print(waiting_list[0])
for tr in waiting_list:
    match_tu, distance_tu_tr = find_tu(tr)
    ride_dict, available_tu_ts = generate_ride(locals().get(match_tu), tr, ride_dict, distance_tu_tr, current_ts)

In [ ]:
print(Tu._registry)
for tu in Tu._registry:
    print(tu.get_state())